In [27]:
%%markdown

UsageError: %%markdown is a cell magic, but the cell body is empty.


In [1]:
import numpy as np
import pandas as pd

train_set_days = pd.read_csv("train_set_days.csv")
train_set_weeks = pd.read_csv("train_set_weeks.csv")
train_set_sfa = pd.read_csv("train_set_sfa.csv")
info_business_points = pd.read_csv("info_business_points.csv")
info_groups = pd.read_csv("info_groups.csv")
info_items = pd.read_csv("info_items.csv")

test_set_weeks = pd.read_csv('test_set_weeks.csv')

# Preprocessing
## .._set_weeks 

In [2]:
def prepare_set_weeks(df_param):
    df_param.wk = df_param.wk.astype(str)
    df_param.wk = df_param.wk.str[:4] + ',' + df_param.wk.str[4:]
    df_param[['wk_year', 'year']] = df_param.wk.str.split(',', expand=True)
    df_param = df_param.drop('wk', axis=1)
    df_param.wk_year = df_param.wk_year.astype(int)
    df_param.year = df_param.year.astype(int)
    return df_param

In [3]:
train_set_weeks = prepare_set_weeks(train_set_weeks)
train_set_weeks.head(5)

,id,N wk,idFilial,KanalDB,idSubGrp,value,wk_year,year
0,0,1,9,2,3,3560.0,2014,1
1,1,2,9,2,3,7120.0,2014,2
2,2,3,9,2,3,57672.0,2014,3
3,3,4,9,2,3,37380.0,2014,4
4,4,5,9,2,3,80990.0,2014,5


In [4]:
train = train_set_weeks.drop(['id','value'], axis=1)
target = train_set_weeks.value

In [5]:
test_set_weeks = prepare_set_weeks(test_set_weeks)
test_set_weeks.head(5)

,id,N wk,idFilial,KanalDB,idSubGrp,wk_year,year
0,0,184,9,2,3,2017,29
1,1,185,9,2,3,2017,30
2,2,186,9,2,3,2017,31
3,3,187,9,2,3,2017,32
4,4,188,9,2,3,2017,33


In [6]:
test = test_set_weeks.drop(['id'], axis=1)

## Set SFA

In [57]:
def prepare_set_sfa(df_param):
    notnull_idxs = pd.notnull(df_param.calc_share)
    df_param.calc_share[notnull_idxs] = df_param.calc_share[notnull_idxs].astype(str)
    df_param.calc_share[notnull_idxs] = df_param.calc_share[notnull_idxs].str[:-1]
    df_param.calc_share = df_param.calc_share.astype(float)
    df_param.calc_share /= 100
    df_param.visit_date = pd.to_datetime(df_param.visit_date, format='%d.%m.%Y')
    df_param['day'] = df_param.visit_date.dt.day
    df_param['month'] = df_param.visit_date.dt.month
    df_param['year'] = df_param.visit_date.dt.year
    df_param = df_param.drop('visit_date', axis=1)
    return df_param

In [58]:
train_set_sfa = prepare_set_sfa(train_set_sfa)
train_set_sfa.head(10)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,id,idBP,idFilial,calc_share,visits_count,calls_count,day,month,year
0,0,63936,4,NaN,0,1,1,8,2015
1,1,5339,6,NaN,1,0,1,8,2015
2,2,62341,10,NaN,0,1,1,8,2015
3,3,64341,10,NaN,1,0,1,8,2015
4,4,67239,10,NaN,1,0,1,8,2015
5,5,67344,10,NaN,0,1,1,8,2015
6,6,69475,10,NaN,0,1,1,8,2015
7,7,14262,11,NaN,0,1,1,8,2015
8,8,702,12,NaN,0,1,1,8,2015
9,9,1356,12,NaN,0,1,1,8,2015


# Train + Control
## Spliting

In [11]:
from sklearn.model_selection import train_test_split
TRNtrain, TRNtest, TARtrain, TARtest = train_test_split(train, target, test_size=0.3, random_state=666)

## Models

In [9]:
from catboost import CatBoostRegressor
from sklearn.linear_model import Lasso

models = []
models.append(CatBoostRegressor())
models.append(Lasso(alpha=0.1))

In [12]:
from sklearn.metrics import mean_squared_error

for model in models:
    model.fit(TRNtrain, TARtrain)
    pred = model.predict(TRNtest)
    print('RMSE = ', np.sqrt(mean_squared_error(TARtest, pred)))

0: learn: 122519.4	total: 119ms	remaining: 1m 59s
1: learn: 120508.5	total: 179ms	remaining: 1m 29s
2: learn: 118485.4	total: 229ms	remaining: 1m 16s
3: learn: 116445.8	total: 289ms	remaining: 1m 11s
4: learn: 114805.2	total: 328ms	remaining: 1m 5s
5: learn: 112845.6	total: 377ms	remaining: 1m 2s
6: learn: 111306	total: 416ms	remaining: 59s
7: learn: 109430.1	total: 476ms	remaining: 59s
8: learn: 107737.7	total: 526ms	remaining: 57.9s
9: learn: 106096.3	total: 586ms	remaining: 58s
10: learn: 104616.9	total: 625ms	remaining: 56.2s
11: learn: 103011.5	total: 672ms	remaining: 55.3s
12: learn: 101720.3	total: 718ms	remaining: 54.5s
13: learn: 100303.8	total: 759ms	remaining: 53.5s
14: learn: 98980.56	total: 805ms	remaining: 52.9s
15: learn: 97821.52	total: 833ms	remaining: 51.2s
16: learn: 96650.55	total: 880ms	remaining: 50.9s
17: learn: 95580.72	total: 920ms	remaining: 50.2s
18: learn: 94514.29	total: 957ms	remaining: 49.4s
19: learn: 93481.7	total: 997ms	remaining: 48.9s
20: learn: 9250

167: learn: 61029.8	total: 7.05s	remaining: 34.9s
168: learn: 61002.29	total: 7.09s	remaining: 34.9s
169: learn: 60923.23	total: 7.16s	remaining: 34.9s
170: learn: 60879.07	total: 7.2s	remaining: 34.9s
171: learn: 60852.87	total: 7.23s	remaining: 34.8s
172: learn: 60788.5	total: 7.28s	remaining: 34.8s
173: learn: 60776.16	total: 7.35s	remaining: 34.9s
174: learn: 60740.9	total: 7.41s	remaining: 34.9s
175: learn: 60714.79	total: 7.45s	remaining: 34.9s
176: learn: 60674.07	total: 7.5s	remaining: 34.9s
177: learn: 60640.37	total: 7.54s	remaining: 34.8s
178: learn: 60592.54	total: 7.58s	remaining: 34.8s
179: learn: 60547.76	total: 7.64s	remaining: 34.8s
180: learn: 60521.04	total: 7.71s	remaining: 34.9s
181: learn: 60457.58	total: 7.78s	remaining: 35s
182: learn: 60399.39	total: 7.83s	remaining: 35s
183: learn: 60354.27	total: 7.86s	remaining: 34.9s
184: learn: 60289.85	total: 7.91s	remaining: 34.8s
185: learn: 60268.1	total: 7.96s	remaining: 34.8s
186: learn: 60205.29	total: 8.01s	remaini

330: learn: 56300.48	total: 14.9s	remaining: 30.2s
331: learn: 56267.98	total: 15s	remaining: 30.2s
332: learn: 56260.5	total: 15s	remaining: 30.1s
333: learn: 56248.03	total: 15.1s	remaining: 30.1s
334: learn: 56215.19	total: 15.1s	remaining: 30s
335: learn: 56184.89	total: 15.2s	remaining: 30s
336: learn: 56173.06	total: 15.2s	remaining: 29.9s
337: learn: 56162.21	total: 15.2s	remaining: 29.8s
338: learn: 56151.42	total: 15.3s	remaining: 29.8s
339: learn: 56129.18	total: 15.3s	remaining: 29.7s
340: learn: 56121.37	total: 15.3s	remaining: 29.6s
341: learn: 56112.74	total: 15.4s	remaining: 29.6s
342: learn: 56101.42	total: 15.4s	remaining: 29.5s
343: learn: 56095.46	total: 15.4s	remaining: 29.5s
344: learn: 56086.62	total: 15.5s	remaining: 29.4s
345: learn: 56082.51	total: 15.5s	remaining: 29.3s
346: learn: 56050.89	total: 15.6s	remaining: 29.3s
347: learn: 56042.52	total: 15.6s	remaining: 29.2s
348: learn: 56025.27	total: 15.6s	remaining: 29.2s
349: learn: 55992.29	total: 15.7s	remain

494: learn: 54248.27	total: 21.4s	remaining: 21.9s
495: learn: 54234.89	total: 21.5s	remaining: 21.8s
496: learn: 54234.85	total: 21.5s	remaining: 21.8s
497: learn: 54230.83	total: 21.6s	remaining: 21.8s
498: learn: 54225.11	total: 21.6s	remaining: 21.7s
499: learn: 54225.1	total: 21.7s	remaining: 21.7s
500: learn: 54225.08	total: 21.7s	remaining: 21.6s
501: learn: 54218.32	total: 21.8s	remaining: 21.6s
502: learn: 54210.57	total: 21.8s	remaining: 21.5s
503: learn: 54195.38	total: 21.8s	remaining: 21.5s
504: learn: 54190.67	total: 21.9s	remaining: 21.4s
505: learn: 54186.81	total: 21.9s	remaining: 21.4s
506: learn: 54181.49	total: 21.9s	remaining: 21.3s
507: learn: 54164.13	total: 22s	remaining: 21.3s
508: learn: 54161.24	total: 22s	remaining: 21.2s
509: learn: 54144.14	total: 22s	remaining: 21.2s
510: learn: 54133.34	total: 22.1s	remaining: 21.1s
511: learn: 54117.21	total: 22.1s	remaining: 21.1s
512: learn: 54114.04	total: 22.2s	remaining: 21.1s
513: learn: 54114.03	total: 22.2s	rema

661: learn: 53071.43	total: 28.2s	remaining: 14.4s
662: learn: 53068.95	total: 28.3s	remaining: 14.4s
663: learn: 53060.82	total: 28.3s	remaining: 14.3s
664: learn: 53046.8	total: 28.4s	remaining: 14.3s
665: learn: 53033.47	total: 28.4s	remaining: 14.3s
666: learn: 53029.71	total: 28.5s	remaining: 14.2s
667: learn: 53026.28	total: 28.5s	remaining: 14.2s
668: learn: 53021.83	total: 28.5s	remaining: 14.1s
669: learn: 53010.23	total: 28.6s	remaining: 14.1s
670: learn: 53010.23	total: 28.6s	remaining: 14s
671: learn: 53004.82	total: 28.7s	remaining: 14s
672: learn: 53002.84	total: 28.7s	remaining: 14s
673: learn: 52978	total: 28.8s	remaining: 13.9s
674: learn: 52978	total: 28.8s	remaining: 13.9s
675: learn: 52973.02	total: 28.9s	remaining: 13.8s
676: learn: 52973.02	total: 28.9s	remaining: 13.8s
677: learn: 52973.02	total: 29s	remaining: 13.8s
678: learn: 52973.02	total: 29.1s	remaining: 13.7s
679: learn: 52973.02	total: 29.1s	remaining: 13.7s
680: learn: 52968.9	total: 29.2s	remaining: 13

823: learn: 52261.58	total: 34.8s	remaining: 7.44s
824: learn: 52261.05	total: 34.9s	remaining: 7.4s
825: learn: 52253.88	total: 34.9s	remaining: 7.36s
826: learn: 52250.17	total: 35s	remaining: 7.32s
827: learn: 52245.73	total: 35s	remaining: 7.27s
828: learn: 52242.65	total: 35.1s	remaining: 7.23s
829: learn: 52236.66	total: 35.1s	remaining: 7.19s
830: learn: 52225.3	total: 35.2s	remaining: 7.15s
831: learn: 52221.16	total: 35.2s	remaining: 7.11s
832: learn: 52221.16	total: 35.3s	remaining: 7.07s
833: learn: 52217.95	total: 35.3s	remaining: 7.03s
834: learn: 52205.64	total: 35.4s	remaining: 6.99s
835: learn: 52197.66	total: 35.4s	remaining: 6.94s
836: learn: 52194.12	total: 35.4s	remaining: 6.9s
837: learn: 52194.12	total: 35.5s	remaining: 6.85s
838: learn: 52194.12	total: 35.5s	remaining: 6.81s
839: learn: 52194.12	total: 35.5s	remaining: 6.76s
840: learn: 52192.04	total: 35.6s	remaining: 6.72s
841: learn: 52173.95	total: 35.6s	remaining: 6.68s
842: learn: 52171.26	total: 35.6s	rema

986: learn: 51632.71	total: 41.6s	remaining: 549ms
987: learn: 51629.08	total: 41.7s	remaining: 506ms
988: learn: 51627.6	total: 41.7s	remaining: 464ms
989: learn: 51623.17	total: 41.8s	remaining: 422ms
990: learn: 51623.17	total: 41.8s	remaining: 380ms
991: learn: 51623.17	total: 41.8s	remaining: 337ms
992: learn: 51620.59	total: 41.9s	remaining: 295ms
993: learn: 51618.52	total: 41.9s	remaining: 253ms
994: learn: 51614.66	total: 42s	remaining: 211ms
995: learn: 51603.71	total: 42s	remaining: 169ms
996: learn: 51603.71	total: 42s	remaining: 127ms
997: learn: 51603.4	total: 42.1s	remaining: 84.3ms
998: learn: 51602.78	total: 42.1s	remaining: 42.2ms
999: learn: 51596.46	total: 42.2s	remaining: 0us
RMSE =  51628.6903395
RMSE =  87064.0041699


C:\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


## Submission

In [ ]:
model_number = 0
clf.best_estimator_.fit(train, target)
pred = clf.best_estimator_.predict(test)
pd.DataFrame({'id': test_set_weeks.id, 'value': pred}).to_csv('second_subm_cb.csv')

# Alpha on LASSO

In [40]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso

param_grid = { 'alpha':[10**i for i in range(-4,3)] }
clf = GridSearchCV(Lasso(), param_grid=param_grid, scoring='neg_mean_squared_error').fit(TRNtrain, TARtrain)
pred = clf.best_estimator_.predict(TRNtest)
print("RMSE=", np.sqrt(mean_squared_error(TARtest, pred)))
print(np.sqrt(-clf.best_score_ ))
print(clf.best_params_)

C:\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with v

RMSE= 87063.9659417
92231.1047719
{'alpha': 0.0001}


C:\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [42]:
from sklearn.model_selection import cross_val_predict

pred = cross_val_predict(clf.best_estimator_, TRNtrain, TRNtarget)
error_obj = np.abs(pred - TRNtarget)

C:\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [45]:
import matplotlib.pyplot as plt

plt.grid()
plt.ylabel('Errors')
plt.xlabel('Numbers of objects')

error_obj_sorted = np.sort(error_obj)
plt.bar( [i for i in range(len(error_obj_sorted))], error_obj_sorted )
plt.show()

### Attempt to train on ejections(unlucky)

In [54]:
idx_more = error_obj > 350000
idx_less = error_obj < 50000
clf.best_estimator_.fit(TRNtrain[idx_more | idx_less], TARtrain[idx_more | idx_less])
pred = clf.best_estimator_.predict(TRNtest)
print("RMSE=", np.sqrt(mean_squared_error(TARtest, pred)))

RMSE= 88264.7508557


C:\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


## GridSearch CatBoostRegressor

In [59]:
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostRegressor

param_grid = {'iterations' : [2000]}
clf = GridSearchCV(CatBoostRegressor(), param_grid=param_grid, scoring='neg_mean_squared_error').fit(TRNtrain, TARtrain)
pred = clf.best_estimator_.predict(TRNtest)
print("RMSE=", np.sqrt(mean_squared_error(TARtest, pred)))
print(np.sqrt(-clf.best_score_ ))
print(clf.best_params_)

0: learn: 123540.7	total: 52.8ms	remaining: 1m 45s
1: learn: 121403.7	total: 89.1ms	remaining: 1m 29s
2: learn: 119306.1	total: 163ms	remaining: 1m 48s
3: learn: 117329.8	total: 233ms	remaining: 1m 56s
4: learn: 115339.4	total: 310ms	remaining: 2m 3s
5: learn: 113570.6	total: 387ms	remaining: 2m 8s
6: learn: 111994.1	total: 417ms	remaining: 1m 58s
7: learn: 110551.7	total: 450ms	remaining: 1m 52s
8: learn: 109111.5	total: 486ms	remaining: 1m 47s
9: learn: 107527.7	total: 516ms	remaining: 1m 42s
10: learn: 105831.3	total: 552ms	remaining: 1m 39s
11: learn: 104572.4	total: 582ms	remaining: 1m 36s
12: learn: 103218.6	total: 618ms	remaining: 1m 34s
13: learn: 101855.9	total: 663ms	remaining: 1m 33s
14: learn: 100718.4	total: 692ms	remaining: 1m 31s
15: learn: 99360.45	total: 728ms	remaining: 1m 30s
16: learn: 98200.9	total: 758ms	remaining: 1m 28s
17: learn: 97113.07	total: 794ms	remaining: 1m 27s
18: learn: 96108.82	total: 824ms	remaining: 1m 25s
19: learn: 94787.82	total: 861ms	remaining

162: learn: 59735.83	total: 6.7s	remaining: 1m 15s
163: learn: 59687.27	total: 6.73s	remaining: 1m 15s
164: learn: 59658.43	total: 6.76s	remaining: 1m 15s
165: learn: 59614.14	total: 6.81s	remaining: 1m 15s
166: learn: 59570.04	total: 6.87s	remaining: 1m 15s
167: learn: 59543.83	total: 6.9s	remaining: 1m 15s
168: learn: 59484.22	total: 6.93s	remaining: 1m 15s
169: learn: 59431.34	total: 6.97s	remaining: 1m 15s
170: learn: 59353.18	total: 7.01s	remaining: 1m 14s
171: learn: 59316.92	total: 7.06s	remaining: 1m 14s
172: learn: 59306.83	total: 7.08s	remaining: 1m 14s
173: learn: 59249.64	total: 7.13s	remaining: 1m 14s
174: learn: 59217.02	total: 7.19s	remaining: 1m 14s
175: learn: 59149.09	total: 7.23s	remaining: 1m 14s
176: learn: 59064.04	total: 7.26s	remaining: 1m 14s
177: learn: 58999.26	total: 7.3s	remaining: 1m 14s
178: learn: 58970.15	total: 7.41s	remaining: 1m 15s
179: learn: 58928.54	total: 7.46s	remaining: 1m 15s
180: learn: 58880.9	total: 7.49s	remaining: 1m 15s
181: learn: 5881

323: learn: 54804.98	total: 13.7s	remaining: 1m 10s
324: learn: 54791.55	total: 13.7s	remaining: 1m 10s
325: learn: 54775.36	total: 13.7s	remaining: 1m 10s
326: learn: 54757.05	total: 13.8s	remaining: 1m 10s
327: learn: 54743.08	total: 13.8s	remaining: 1m 10s
328: learn: 54700.8	total: 13.9s	remaining: 1m 10s
329: learn: 54677.92	total: 13.9s	remaining: 1m 10s
330: learn: 54654.02	total: 13.9s	remaining: 1m 10s
331: learn: 54633.01	total: 13.9s	remaining: 1m 10s
332: learn: 54632.99	total: 14s	remaining: 1m 10s
333: learn: 54613.83	total: 14s	remaining: 1m 9s
334: learn: 54606.55	total: 14.1s	remaining: 1m 9s
335: learn: 54582.01	total: 14.1s	remaining: 1m 9s
336: learn: 54576.76	total: 14.1s	remaining: 1m 9s
337: learn: 54520.44	total: 14.2s	remaining: 1m 9s
338: learn: 54505.14	total: 14.2s	remaining: 1m 9s
339: learn: 54468.57	total: 14.2s	remaining: 1m 9s
340: learn: 54448.68	total: 14.3s	remaining: 1m 9s
341: learn: 54448.67	total: 14.3s	remaining: 1m 9s
342: learn: 54442.05	total

486: learn: 52495.4	total: 20s	remaining: 1m 2s
487: learn: 52483.01	total: 20.1s	remaining: 1m 2s
488: learn: 52469.66	total: 20.2s	remaining: 1m 2s
489: learn: 52459.91	total: 20.2s	remaining: 1m 2s
490: learn: 52449.91	total: 20.2s	remaining: 1m 2s
491: learn: 52441.73	total: 20.3s	remaining: 1m 2s
492: learn: 52436	total: 20.4s	remaining: 1m 2s
493: learn: 52429.6	total: 20.4s	remaining: 1m 2s
494: learn: 52425.73	total: 20.5s	remaining: 1m 2s
495: learn: 52416.18	total: 20.6s	remaining: 1m 2s
496: learn: 52408.42	total: 20.6s	remaining: 1m 2s
497: learn: 52398.12	total: 20.6s	remaining: 1m 2s
498: learn: 52383.81	total: 20.7s	remaining: 1m 2s
499: learn: 52359.61	total: 20.8s	remaining: 1m 2s
500: learn: 52357.25	total: 20.8s	remaining: 1m 2s
501: learn: 52345.5	total: 20.8s	remaining: 1m 2s
502: learn: 52341.56	total: 20.9s	remaining: 1m 2s
503: learn: 52329.77	total: 20.9s	remaining: 1m 2s
504: learn: 52323.87	total: 20.9s	remaining: 1m 2s
505: learn: 52311.76	total: 21s	remaini

650: learn: 51107.17	total: 26.7s	remaining: 55.4s
651: learn: 51106.51	total: 26.7s	remaining: 55.3s
652: learn: 51106.51	total: 26.8s	remaining: 55.2s
653: learn: 51099.95	total: 26.8s	remaining: 55.2s
654: learn: 51080.06	total: 26.8s	remaining: 55.1s
655: learn: 51071.14	total: 26.9s	remaining: 55s
656: learn: 51067.95	total: 26.9s	remaining: 55s
657: learn: 51056.61	total: 26.9s	remaining: 54.9s
658: learn: 51047.49	total: 27s	remaining: 54.9s
659: learn: 51047.11	total: 27s	remaining: 54.8s
660: learn: 51044.95	total: 27s	remaining: 54.8s
661: learn: 51044.94	total: 27.1s	remaining: 54.7s
662: learn: 51039.19	total: 27.1s	remaining: 54.7s
663: learn: 51034.69	total: 27.2s	remaining: 54.7s
664: learn: 51034.67	total: 27.2s	remaining: 54.7s
665: learn: 51028.43	total: 27.3s	remaining: 54.6s
666: learn: 51023.7	total: 27.3s	remaining: 54.6s
667: learn: 51020.31	total: 27.4s	remaining: 54.6s
668: learn: 51009.43	total: 27.4s	remaining: 54.5s
669: learn: 51005.81	total: 27.5s	remainin

...

1305: learn: 49525.91	total: 59.8s	remaining: 31.8s
1306: learn: 49523.68	total: 59.8s	remaining: 31.7s
1307: learn: 49521.58	total: 59.9s	remaining: 31.7s
1308: learn: 49521.52	total: 59.9s	remaining: 31.6s
1309: learn: 49520.4	total: 60s	remaining: 31.6s
1310: learn: 49515.4	total: 60s	remaining: 31.5s
1311: learn: 49514.18	total: 1m	remaining: 31.5s
1312: learn: 49512.06	total: 1m	remaining: 31.4s
1313: learn: 49509.61	total: 1m	remaining: 31.4s
1314: learn: 49508.59	total: 1m	remaining: 31.3s
1315: learn: 49508.32	total: 1m	remaining: 31.3s
1316: learn: 49508.32	total: 1m	remaining: 31.2s
1317: learn: 49507.84	total: 1m	remaining: 31.2s
1318: learn: 49507.84	total: 1m	remaining: 31.1s
1319: learn: 49506.7	total: 1m	remaining: 31.1s
1320: learn: 49506.08	total: 1m	remaining: 31s
1321: learn: 49506.08	total: 1m	remaining: 31s
1322: learn: 49494.96	total: 1m	remaining: 30.9s
1323: learn: 49492.89	total: 1m	remaining: 30.9s
1324: learn: 49483.04	total: 1m	remaining: 30.8s
1325: learn: 

1468: learn: 49162.17	total: 1m 8s	remaining: 24.9s
1469: learn: 49162.03	total: 1m 8s	remaining: 24.8s
1470: learn: 49162.03	total: 1m 8s	remaining: 24.8s
1471: learn: 49161.8	total: 1m 9s	remaining: 24.8s
1472: learn: 49159.91	total: 1m 9s	remaining: 24.7s
1473: learn: 49159.91	total: 1m 9s	remaining: 24.7s
1474: learn: 49159.84	total: 1m 9s	remaining: 24.6s
1475: learn: 49159.2	total: 1m 9s	remaining: 24.6s
1476: learn: 49155.21	total: 1m 9s	remaining: 24.5s
1477: learn: 49154.87	total: 1m 9s	remaining: 24.5s
1478: learn: 49152.08	total: 1m 9s	remaining: 24.5s
1479: learn: 49151.34	total: 1m 9s	remaining: 24.4s
1480: learn: 49142.92	total: 1m 9s	remaining: 24.4s
1481: learn: 49139.16	total: 1m 9s	remaining: 24.3s
1482: learn: 49138.49	total: 1m 9s	remaining: 24.3s
1483: learn: 49138.48	total: 1m 9s	remaining: 24.3s
1484: learn: 49137.5	total: 1m 9s	remaining: 24.2s
1485: learn: 49136.93	total: 1m 9s	remaining: 24.2s
1486: learn: 49132.74	total: 1m 9s	remaining: 24.1s
1487: learn: 49

1627: learn: 48895.76	total: 1m 19s	remaining: 18.1s
1628: learn: 48895.45	total: 1m 19s	remaining: 18s
1629: learn: 48895.39	total: 1m 19s	remaining: 18s
1630: learn: 48894.88	total: 1m 19s	remaining: 17.9s
1631: learn: 48894.84	total: 1m 19s	remaining: 17.9s
1632: learn: 48892.56	total: 1m 19s	remaining: 17.8s
1633: learn: 48891.54	total: 1m 19s	remaining: 17.8s
1634: learn: 48888.46	total: 1m 19s	remaining: 17.7s
1635: learn: 48887.43	total: 1m 19s	remaining: 17.7s
1636: learn: 48887.38	total: 1m 19s	remaining: 17.6s
1637: learn: 48887.21	total: 1m 19s	remaining: 17.6s
1638: learn: 48887.14	total: 1m 19s	remaining: 17.6s
1639: learn: 48886.93	total: 1m 19s	remaining: 17.5s
1640: learn: 48886.91	total: 1m 19s	remaining: 17.5s
1641: learn: 48886.21	total: 1m 19s	remaining: 17.4s
1642: learn: 48886.12	total: 1m 19s	remaining: 17.4s
1643: learn: 48881.65	total: 1m 19s	remaining: 17.3s
1644: learn: 48880.4	total: 1m 20s	remaining: 17.3s
1645: learn: 48875.39	total: 1m 20s	remaining: 17.2

1786: learn: 48658.8	total: 1m 28s	remaining: 10.6s
1787: learn: 48656.45	total: 1m 29s	remaining: 10.6s
1788: learn: 48656.41	total: 1m 29s	remaining: 10.5s
1789: learn: 48655.87	total: 1m 29s	remaining: 10.5s
1790: learn: 48653.75	total: 1m 29s	remaining: 10.4s
1791: learn: 48650.45	total: 1m 29s	remaining: 10.4s
1792: learn: 48649.99	total: 1m 29s	remaining: 10.3s
1793: learn: 48649.83	total: 1m 29s	remaining: 10.3s
1794: learn: 48643.77	total: 1m 29s	remaining: 10.2s
1795: learn: 48643.76	total: 1m 29s	remaining: 10.2s
1796: learn: 48643.62	total: 1m 29s	remaining: 10.1s
1797: learn: 48643.48	total: 1m 29s	remaining: 10.1s
1798: learn: 48642.94	total: 1m 29s	remaining: 10s
1799: learn: 48642.05	total: 1m 29s	remaining: 10s
1800: learn: 48640.7	total: 1m 30s	remaining: 9.95s
1801: learn: 48636.55	total: 1m 30s	remaining: 9.9s
1802: learn: 48635.92	total: 1m 30s	remaining: 9.85s
1803: learn: 48635.91	total: 1m 30s	remaining: 9.8s
1804: learn: 48635.9	total: 1m 30s	remaining: 9.75s
18

1944: learn: 48445.84	total: 1m 39s	remaining: 2.8s
1945: learn: 48445.76	total: 1m 39s	remaining: 2.75s
1946: learn: 48441.92	total: 1m 39s	remaining: 2.7s
1947: learn: 48441.47	total: 1m 39s	remaining: 2.65s
1948: learn: 48441.11	total: 1m 39s	remaining: 2.6s
1949: learn: 48432.92	total: 1m 39s	remaining: 2.55s
1950: learn: 48432.83	total: 1m 39s	remaining: 2.5s
1951: learn: 48432.71	total: 1m 39s	remaining: 2.45s
1952: learn: 48432.28	total: 1m 39s	remaining: 2.4s
1953: learn: 48431.28	total: 1m 39s	remaining: 2.35s
1954: learn: 48429.76	total: 1m 39s	remaining: 2.29s
1955: learn: 48425.4	total: 1m 39s	remaining: 2.24s
1956: learn: 48425.29	total: 1m 39s	remaining: 2.19s
1957: learn: 48422.18	total: 1m 39s	remaining: 2.14s
1958: learn: 48421.53	total: 1m 39s	remaining: 2.09s
1959: learn: 48421.05	total: 1m 40s	remaining: 2.04s
1960: learn: 48421.05	total: 1m 40s	remaining: 1.99s
1961: learn: 48421.04	total: 1m 40s	remaining: 1.94s
1962: learn: 48420.7	total: 1m 40s	remaining: 1.89s
